### 1. Load file and pre-processing

In [1]:
!pip install --quiet cornac==1.12.0
!pip install spacy_langdetect

     |████████████████████████████████| 13.4 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 998 kB 4.9 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993431 sha256=89756bbdcd16fb503dfd779963da4950a2ffbc08046269752486189d1b166921
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect


In [6]:
# !pip install recommenders
!pip install recmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 38.1 MB 369 kB/s 
     |████████████████████████████████| 793 kB 62.7 MB/s 
     |████████████████████████████████| 13.2 MB 30.8 MB/s 
     |████████████████████████████████| 11.2 MB 49.8 MB/s 
     |████████████████████████████████| 381 kB 74.3 MB/s 
     |████████████████████████████████| 930 kB 54.6 MB/s 
     |████████████████████████████████| 208 kB 75.4 MB/s 
     |████████████████████████████████| 297 kB 74.4 MB/s 
     |████████████████████████████████| 232 kB 74.0 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 138 kB 74.9 MB/s 
     |████████████████████████████████| 48 kB 6.1 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 51 kB 8.2 MB/s 
     |████████████████████████████████| 4.0 MB 46.2 MB/s 
     |███████████████████████████████

In [7]:
%tensorflow_version 1.x

import cornac
from cornac.data import Reader
from cornac.datasets import citeulike
from cornac.eval_methods import RatioSplit, BaseMethod
from cornac.data import TextModality, ImageModality
from cornac.data.text import BaseTokenizer

import pandas as pd
from string import digits
# from spacy_langdetect import LanguageDetector
# from spacy.language import Language

import spacy

import pickle
import cloudpickle

SEED = 42


TensorFlow 1.x selected.


In [ ]:
# def get_lang_detector(nlp, name):
#     return LanguageDetector()

# nlp = spacy.load('en_core_web_sm')  # 1
# Language.factory("language_detector", func=get_lang_detector)
# nlp.add_pipe('language_detector', last=True)

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
folder_path = '/content/drive/MyDrive/rm_project'

review = pd.read_csv(folder_path + '/yelp_sample/final_review_pa.csv', encoding = 'latin1')

train = pd.read_csv(folder_path + '/yelp_sample/test_0.3.csv', encoding = 'latin1')
test = pd.read_csv(folder_path + '/yelp_sample/train_0.7.csv', encoding = 'latin1')

lang = pd.read_csv(folder_path + '/yelp_sample/lang.csv')
hotwords = pd.read_csv(folder_path + '/yelp_sample/hotwords.csv')



In [ ]:
# no. of reviews NOT in english
(lang['lang'] != 'en').sum()

351

In [ ]:
print(train.shape)
print(test.shape)

(411142, 22)
(177128, 22)


In [ ]:
# business = pd.read_csv(folder_path + '/yelp_sample/business.csv')
# business['categories'] = business['categories'].str.lower()
# business.head(2)

In [ ]:
interest = pd.read_csv(folder_path + '/yelp_sample/interest.csv').rename(columns = {'categories': 'interest'})
interest['interest'] = interest['interest'].str.lower()
interest.head(2)

,Unnamed: 0,index,user_id,interest
0,0,0,--2vR0DIsmQ6WfcSzKWigw,public services & government landmarks &...
1,1,1,--4AjktZiHowEIBCMd4CZA,bars nightlife whiskey bars burgers ...


In [ ]:
# merge categories_sum information with train, test df
review = review.merge(interest, how = 'left', on = 'user_id')
train = train.merge(interest, how = 'left', on = 'user_id')
test = test.merge(interest, how = 'left', on = 'user_id')

In [ ]:
# merge hotwords information with train, test df
review = review.merge(hotwords, how = 'left', on = 'review_id')
train = train.merge(hotwords, how = 'left', on = 'review_id')
test = test.merge(hotwords, how = 'left', on = 'review_id')

In [ ]:
# merge language information with train, test df
review = review.merge(lang, how = 'left', on = 'review_id')
train = train.merge(lang, how = 'left', on = 'review_id')
test = test.merge(lang, how = 'left', on = 'review_id')

In [ ]:
# drop records that are not english

# review = review[review['lang'] == 'en']
# train = train[train['lang'] == 'en']
# test = test[test['lang'] == 'en']

In [ ]:
train.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,attributes,categories,hours,Unnamed: 0_x,index,interest,Unnamed: 0_y,hotwords,Unnamed: 0,lang
0,-TDAxzdIkWxxqiVgW9Xf1A,--2vR0DIsmQ6WfcSzKWigw,Jw-ENQw-OGGwpM3nA3ywqw,2.0,72,60,69,Mit unseren British Airways PrÃÂÃÂ¤mienpunk...,2014-07-26 11:51:32,Best Western Plus Philadelphia Airport South,...,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':...","Hotels, Event Planning & Services, Hotels & Tr...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",0,0,public services & government landmarks &...,118970,und der das war wir nicht ein western hotel au...,118970,de
1,pVBwSCJDW59uf4OIz-dO7g,--2vR0DIsmQ6WfcSzKWigw,iqJ3t9iq0fsWaYL93s8Vmw,5.0,68,57,64,"Als wir in Philadelphia waren, haben wir auch ...",2014-07-28 10:49:48,King of Prussia,...,"{'BusinessAcceptsCreditCards': 'False', 'Resta...","Shopping Centers, Fashion, Department Stores, ...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",0,0,public services & government landmarks &...,402296,king prussia das der und den man noch von eink...,402296,de
2,c_DRiA8CPghSk8PR96K2Kg,--2vR0DIsmQ6WfcSzKWigw,krCwF4raTYvBcCEwcbA88Q,4.0,67,56,59,Nach einem verunglÃÂÃÂ¼ckten Besuch der Ger...,2014-07-29 11:10:14,Philadelphia Mills,...,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...","Shopping Centers, Shopping, Restaurants, Fashi...","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",0,0,public services & government landmarks &...,239462,mall der einen die man ein store ist den fãâ¼...,239462,de
3,yUF1T08DTuAxt_J24pGr0g,--2vR0DIsmQ6WfcSzKWigw,6yR6piTIuUG_9P_ctivxKw,5.0,76,67,73,In das MacyÃÂÃÂ´s in Philadelphia sind wir ...,2014-11-07 09:49:15,Macy's,...,"{'BikeParking': 'True', 'GoodForKids': 'True',...","Fashion, Shopping, Men's Clothing, Department ...","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",0,0,public services & government landmarks &...,109997,das der ein und wir von man grand court nur ei...,109997,de
4,-wGzE2pTOZY5SGsEewobaw,--2vR0DIsmQ6WfcSzKWigw,IBIp-yVFl2PxYS-rgc19Yg,5.0,107,93,104,"Wer Philadelphia besucht, um Amerikas historis...",2015-02-02 05:25:46,Independence Visitor Center,...,"{'GoodForKids': 'True', 'RestaurantsDelivery':...","Breakfast & Brunch, Landmarks & Historical Bui...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-17:0', '...",0,0,public services & government landmarks &...,352151,der und das center von man sich visitor mit hi...,352151,de


#### the below text detection was only run once, and results was saved afterwards

In [ ]:
# # example of language detection
# text = review['text'].loc[0]
# doc = nlp(text)
# print(doc._.language['language'])

In [ ]:
# # to detect rows where text is not in English
# lang = review['text'].apply(lambda x: nlp(x)._.language['language'])

# print('Number of reviews NOT in english: {}'.format((lang != 'en').sum()))

In [ ]:
# # filter out non-english reviews
# print('before filter language: ', review.shape)

# review = review[(lang == 'en')]
# print('after filter language: ', review.shape)

In [ ]:
# lang_train = train['text'].apply(lambda x: nlp(x)._.language['language'])

In [ ]:
# lang_test = test['text'].apply(lambda x: nlp(x)._.language['language'])
# print('Number of reviews NOT in english in TEST set: {}'.format((lang_test != 'en').sum()))

In [ ]:
review['text_int'] = review['text'] + review['interest']
train['text_int'] = train['text'] + train['interest']
test['text_int'] = test['text'] + test['interest']


review['text_int_hot'] = review['text'] + review['interest'] + review['hotwords']
train['text_int_hot'] = train['text'] + train['interest'] + train['hotwords']
test['text_int_hot'] = test['text'] + test['interest'] + test['hotwords']


review = review.astype({"text_int_hot": str, "hotwords": str})
train = train.astype({"text_int_hot": str, "hotwords": str})
test = test.astype({"text_int_hot": str, "hotwords": str})


In [ ]:
# use full review df
ratings_df = review[['user_id', 'business_id', 'stars']].to_numpy()
# ratings = ratings_df.values.tolist()

# use splited train & test
ratings_train = train[['user_id', 'business_id', 'stars_x']].to_numpy()
# ratings_train_list = ratings_train.values.tolist()

ratings_test = test[['user_id', 'business_id', 'stars_x']].to_numpy()
# ratings_test_list = ratings_test.values.tolist()


In [ ]:
len(ratings_train)

411142

In [ ]:
# use full review df
texts = review['text'].to_list()
texts_int = review['text_int'].to_list()
texts_int_hot = review['text_int_hot'].to_list()
texts_hot = review['hotwords'].to_list()


# use splited train & test
texts_train = train['text'].to_list()
texts_int_train = train['text_int'].to_list()
texts_int_hot_train = train['text_int_hot'].to_list()
texts_hot_train = train['hotwords'].to_list()


texts_test = test['text'].to_list()
texts_int_test = test['text_int'].to_list()
texts_int_hot_test = test['text_int_hot'].to_list()
texts_hot_test = test['hotwords'].to_list()


In [ ]:
_# use full review df
ids = review['business_id'].to_list()

# use splited train & test
ids_train = train['business_id'].to_list()
ids_test = test['business_id'].to_list()

### 2. Model dev

In [ ]:
# Instantiate a TextModality, it makes it convenient to work with text auxiliary information
# For more details, please refer to the tutorial on how to work with auxiliary data
item_text_modality = TextModality(
    corpus=texts, # to select text data used
    ids=ids,
    tokenizer=BaseTokenizer(stop_words="english"),
    max_vocab=8000,
    max_doc_freq=0.5,
)

In [ ]:
# base method
base_method = BaseMethod.from_splits(
    ratings_train,
    ratings_test,
    rating_threshold = 3.0,
    item_text=item_text_modality,
    verbose=True,
    seed=SEED
)

rating_threshold = 3.0
exclude_unknowns = False


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 14541 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 26177
Number of items = 30677
Number of ratings = 396601
Max rating = 5.0
Min rating = 1.0
Global mean = 3.8


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 3872 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Test data:
Number of users = 22115
Number of items = 25184
Number of ratings = 173256
Number of unknown users = 586
Number of unknown items = 1546
---
Total users = 26763
Total items = 32223


In [ ]:
# # ratio split
# ratio_split = RatioSplit(
#     data=ratings_df.to_numpy(),
#     test_size=0.2,
#     exclude_unknowns=True,
#     item_text=item_text_modality,
#     verbose=True,
#     seed=123,
#     rating_threshold=3,
# )

In [ ]:
eval_k = 10

rmse = cornac.metrics.RMSE()
mae = cornac.metrics.MAE()

f1 = cornac.metrics.FMeasure(k=eval_k)
recall = cornac.metrics.Recall(k = eval_k)
precision = cornac.metrics.Precision(k = eval_k)
ndcg = cornac.metrics.NDCG(k=eval_k)
ncrr = cornac.metrics.NCRR(k=eval_k)

auc = cornac.metrics.AUC()
map = cornac.metrics.MAP()
mrr = cornac.metrics.MRR()



metrics = [rmse, mae, f1, recall, precision, ndcg, ncrr, auc, map, mrr]

In [ ]:

cdl = cornac.models.CDL(
    k=100,
    autoencoder_structure=[200],
    max_iter=50,
    lambda_u=0.1,
    lambda_v=1,
    lambda_w=0.1,
    lambda_n=1000,
    seed=SEED
)


# Put everything together into an experiment and run it
cornac.Experiment(eval_method=base_method, models=[cdl], metrics=metrics).run()


[CDL] Training started!


  0%|          | 0/50 [00:00<?, ?it/s]

Learning completed!

[CDL] Evaluation started!


Rating:   0%|          | 0/173256 [00:00<?, ?it/s]

Ranking:   0%|          | 0/22115 [00:00<?, ?it/s]


TEST:
...
    |    MAE |   RMSE |    AUC |  F1@10 |    MAP |    MRR | NCRR@10 | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CDL | 2.3333 | 2.6025 | 0.8245 | 0.0334 | 0.0327 | 0.1034 |  0.0413 |  0.0453 |       0.0296 |    0.0518 | 1181.4961 | 283.1331



In [12]:
# folder_path
model_path = '/models'
model_name = '/cdl_70_text_de.pkl'



In [ ]:
# save model
cloudpickle.dump(cdl, open(folder_path + model_path + model_name, 'wb'))

In [ ]:
ctr_100 = cornac.models.CTR(k=100, max_iter=50, a=1.0, b=0.01, lambda_u=0.01, lambda_v=0.01, verbose=True, seed=SEED, name = 'ctr_100')

cornac.Experiment(eval_method=base_method, models=[ctr_100], metrics=metrics).run()



[ctr_100] Training started!


  0%|          | 0/50 [00:00<?, ?it/s]

Learning completed!

[ctr_100] Evaluation started!


Rating:   0%|          | 0/173256 [00:00<?, ?it/s]

Ranking:   0%|          | 0/22115 [00:00<?, ?it/s]


TEST:
...
        |    MAE |   RMSE |    AUC |  F1@10 |    MAP |    MRR | NCRR@10 | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------- + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ctr_100 | 2.3849 | 2.6537 | 0.7891 | 0.0299 | 0.0290 | 0.0942 |  0.0365 |  0.0403 |       0.0264 |    0.0466 | 2008.8499 | 280.0606



In [ ]:
# folder_path
model_path = '/models'
model_name = '/ctr_70_text_de.pkl'

# save model
cloudpickle.dump(ctr_100, open(folder_path + model_path + model_name, 'wb'))

### 3. Ranking and interpration

#### 3.1 Load CDL for evaluation metric calculation

coverage, novelty

In [19]:
import recmetrics

In [38]:
# function to recommend for single uid
def output_top(model, uid, top_k = 10):
    user_id = list(model.train_set.user_ids)
    item_id = list(model.train_set.item_ids)

    # print('Start prediction...')    
    item_map = {v:k for k, v in model.train_set.iid_map.items()}

    uidx = user_id.index(uid)
    iidx = model.rank(user_idx = uidx)[0][:top_k]

    result = [item_map[item_idx] for item_idx in iidx]

    return result

In [39]:
# load model
model_name = '/cdl_70_text_de.pkl'
cdl_frompickle = pickle.load(open(folder_path + model_path + model_name, 'rb'))

In [40]:
user_id = list(cdl_frompickle.train_set.user_ids)
catalog = list(cdl_frompickle.train_set.item_ids)

In [41]:
result = pd.DataFrame()

result['user_id'] = user_id
result.head()

,user_id
0,--2vR0DIsmQ6WfcSzKWigw
1,--4AjktZiHowEIBCMd4CZA
2,--_r6E98SNIrGU7weyNxbw
3,--ccVMj2PN6Z9qtdOdlung
4,--dVKamoZnV2vYwKtWMVVA


In [42]:
result['rec@10'] = result['user_id'].apply(lambda x: output_top(cdl_frompickle, x))

In [43]:
result.head(2)

,user_id,rec@10
0,--2vR0DIsmQ6WfcSzKWigw,"[IBIp-yVFl2PxYS-rgc19Yg, kbEVlzQLcYS3JSQPG9QMO..."
1,--4AjktZiHowEIBCMd4CZA,"[LNHq9WxfhN2UBNOR2tnRQQ, wm5mQ4cSpvko9WlCq07RF..."


##### catalog coverage

In [49]:
cat_in_prediction = list(set(result['rec@10'].sum()))

In [53]:
cat_coverage = len(cat_in_prediction) / len(catalog)
print('catalog coverage is {:.2f}'.format(cat_coverage))

catalog coverage is 0.33


##### novelty

In [62]:
nov = review.business_id.value_counts()
pop = dict(nov)

In [65]:
novelty,random_mselfinfo_list = recmetrics.novelty(result['rec@10'], pop, len(user_id), 10)
print('novelty is {:.2f}'.format(cat_coverage))

novelty is 7.43


#### 3.2 Load CTR for prediction intepration

In [17]:
# load model

model_name = '/ctr_base.pkl'

ctr_frompickle = pickle.load(open(folder_path + model_path + model_name, 'rb'))


In [14]:
import numpy as np
from collections import defaultdict

In [18]:
vocab = ctr_frompickle.train_set.item_text.vocab
topic_word_dist = ctr_frompickle.model.beta.T[:, -ctr_frompickle.train_set.item_text.max_vocab:] 
top_word_inds = np.argsort(topic_word_dist, axis=1) + 4  # ingore 4 special tokens

topic_words = {}
topic_df = defaultdict(list)

print("WORD TOPICS:")
for t in range(len(topic_word_dist)):
  top_words = vocab.to_text(top_word_inds[t][-10:][::-1], sep=", ")
  topic_words[t+1] = top_words
  topic_df["Topic"].append(t + 1)
  topic_df["Top words"].append(top_words)
topic_df = pd.DataFrame(topic_df)
topic_df

WORD TOPICS:


,Topic,Top words
0,1,"great, meal, dish, try, looking, place, thing,..."
1,2,"asked, local, variety, thing, place, im, half,..."
2,3,"wanted, business, crowded, cold, location, rud..."
3,4,"hard, old, remember, meal, restaurant, fries, ..."
4,5,"having, started, night, just, doesnt, love, or..."
5,6,"took, quick, dont, going, free, quality, recom..."
6,7,"minutes, came, nice, just, inside, tasty, ask,..."
7,8,"time, like, going, good, chicken, know, said, ..."
8,9,"really, got, say, friendly, place, food, area,..."
9,10,"happy, breakfast, think, restaurant, just, fas..."


In [ ]:
UIDX = 3
TOPK = 5

item_id2idx = ctr_100.train_set.iid_map
item_idx2id = list(ctr_100.train_set.item_ids)

print(f"USER {UIDX} TOP-5 TOPICS:")
topic_df.loc[np.argsort(ctr_100.U[UIDX])[-5:][::-1]]

USER 3 TOP-5 TOPICS:


,Topic,Top words
56,57,"evening, lobster, life, surprisingly, smooth, ..."
82,83,"recommend, stars, decor, basically, wanted, bu..."
47,48,"come, customer, shop, bowl, juice, known, tofu..."
36,37,"sauce, wish, spicy, arrived, hope, woman, char..."
38,39,"ordered, make, check, husband, center, year, e..."


In [ ]:
user_id = list(ctr_100.train_set.user_ids)
item_id = list(ctr_100.train_set.item_ids)
UID = user_id[UIDX]


In [ ]:
# busniess = pd.read_csv(folder_path + '/yelp_sample/business.csv')

In [ ]:
review[review['user_id'] == UID][['text']]

,text
5591,"I've never actually been into Allegro, because..."
31161,The only good thing was the okra with rice. O...
63391,I'm pretty sure there's nothing I could say th...
88327,Please just do it. It's the epitome of Philade...
94502,A friend wanted to go to Sisters and I thought...
129509,I feel bad giving Our Nation's Liberty Bell fo...
184089,My last two birthday cakes have been from Isgr...
212104,Rangoon is great. The only reason that I cann...
244685,If I were to judge Dahlak on its potato salad ...
293483,"I love this place, even though they sometimes ..."


In [ ]:
recommendations, scores = ctr_100.rank(UIDX)
recommendations

array([  73,   75,   69, ..., 3442,  628,  624])

In [ ]:
recommendations, scores = ctr_100.rank(UIDX)

print(f"\nTOP {TOPK} RECOMMENDATIONS FOR USER {UIDX}:")
rec_df = defaultdict(list)
for i in recommendations[:TOPK]:
  rec_df["Business"].append(item_id[i])
  rec_df["Description"].append(ctr_100.train_set.item_text.corpus[i])
pd.DataFrame(rec_df)


TOP 5 RECOMMENDATIONS FOR USER 3:


,Business,Description
0,Biidj3QszVuVO0Q85g5NPw,Banh Minh: 4.5 out of 5\nMy partner and I have...
1,oziGQYLaEWOpNmPKrZP86g,We ordered our wedding cake from isgro and it ...
2,gEX_kEhYgvLKGJtlY45BmQ,"As I am getting ready to order the appetizers,..."
3,14ZGwnDyydXdSBsLXpSUrA,"When the world opened back up, and my partner ..."
4,U30ggGzFpXvc2NZYwOW3qg,I went in with no idea about what I was going ...
